# Map types

In [1]:
import deetly
import os
import pandas as pd
from getpass import getpass
import geopandas as gpd
import pydeck
import altair as alt
import dateutil.parser
import requests
import json

import locale
locale.setlocale(locale.LC_ALL, "no_NO.UTF-8")

'no_NO.UTF-8'

In [2]:
MAPBOX_TOKEN = os.environ['MAPBOX_TOKEN'] 

In [3]:
MAP_URL = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/2020/counties_2020_s_geojson.json'

In [4]:
COLOR_RANGE = [
  [255, 255, 178, 25],
  [254, 217, 118, 85],
  [254, 178, 76, 127],
  [253, 141, 60, 170],
  [240, 59, 32, 212],
  [189, 0, 38, 255]
]

In [5]:
# Create datapackage
description = """
## Skisser til kartvisning for antall søknader om ordinære dagpenger og dagpenger under permittering
"""

package = {
    "name":"Skisser til kartvisning for antall søknader om ordinære dagpenger og dagpenger under permittering", 
    "description":description, 
    "author":"deetly", 
    "theme":"example",
    "license": "MIT",
    "keywords": ["example, deck.gl, pydeck, arbeid, dagpenger, permittering, permitteringer"]
}

dp = deetly.package(package)

Missing mandatory props(s): ['title']
Missing recommended props(s): ['contactPoint', 'distribution', 'keyword', 'publisher', 'spatial', 'temporal']
Property not in DCAT schema: ['name', 'author', 'license', 'keywords']
Invalid type: ['String:name', 'String:description', 'String:author', 'String or list of strings:theme', 'String:license', 'String:keywords', 'ISO date or datetime:issued', 'ISO date or datetime:modified']


# Get data from nav.no

In [6]:
EXCEL_URL = 'https://www.nav.no/no/nav-og-samfunn/statistikk/flere-statistikkomrader/relatert-informasjon/soknader-om-dagpenger/_/attachment/download/411ae583-d0a3-4e5b-9f70-b87e62424518:6f394fda3ad3098d5913a9f4b417a72d5174bea5/2503%20-%20S%C3%B8knader_om_dagpenger.xlsx'

In [7]:
xl_sykefravaer = pd.ExcelFile(EXCEL_URL)
xl_sykefravaer.sheet_names

['Dato',
 'Uke',
 'Fylke. Ordinære',
 'Fylke. Permitterte',
 'Kommune. Permitterte',
 'Bydel. Permitterte',
 'År',
 'Om statistikken']

In [8]:
df_p = pd.read_excel(EXCEL_URL,sheet_name='Kommune. Permitterte', header=6, encoding="latin-1")
df_p.drop(columns=["Unnamed: 0","Unnamed: 2","Unnamed: 7","Unnamed: 8","Unnamed: 9"], axis=1, inplace=True)
df_p.rename(columns={"Unnamed: 1": "Kommune"}, inplace=True)
df_p.drop([0, 0],inplace=True)
df_p.replace({'Kommune': {'Ukjent': '9999 Ukjent'}}, inplace=True)
df_p.replace({'*':0}, inplace=True)
df_p["kommunenummer"] = df_p["Kommune"].apply(lambda x: x[0:4])
df_p["kommune_navn"] = df_p["Kommune"].apply(lambda x: x[4:])
df_p.rename(columns={"I alt": "sum", 11: 'uke11', 12: 'uke12', 13:'uke13'}, inplace=True)
df_p.head(1)

,Kommune,sum,uke11,uke12,uke13,kommunenummer,kommune_navn
1,0301 Oslo,38532,3976,29746,4810,0301,Oslo


In [9]:
df_map = gpd.read_file(MAP_URL)
df_merged = pd.merge(df_map, df_p, how='left', on='kommunenummer')
df_merged['lat'] = df_merged['geometry'].apply(lambda x : round(x.centroid.y,2))
df_merged['lng'] = df_merged['geometry'].apply(lambda x : round(x.centroid.x,2))
df_merged['coordinates']  = df_merged.apply(lambda row: [row['lng'], row['lat']], axis=1)

In [10]:
df_merged.head(1)

,id,kommunenummer,oppdateringsdato,navn,area_kommune,fylke_nummer,fylke_navn,geometry,Kommune,sum,uke11,uke12,uke13,kommune_navn,lat,lng,coordinates
0,0,0301,2019-10-28T00:00:00,Oslo,0.072515,03,OSLO,"POLYGON ((10.68032 60.13353, 10.74694 60.12303...",0301 Oslo,38532,3976,29746,4810,Oslo,59.98,10.74,"[10.74, 59.98]"


In [11]:
# Map centered on Trondheim, Norway
view_state_norway = pydeck.ViewState(
    longitude = 17,
    latitude= 64,
    zoom= 3,
    minZoom= 3,
    maxZoom= 12,
    pitch= 45,
    bearing= 0)

## GeoJson

In [12]:
layer = pydeck.Layer(
    'GeoJsonLayer',
    json.loads(df_merged.to_json()),
    stroked = False,
    filled = True,
    extruded = True,
    wireframe = True,
    elevationScale = 40,
    getElevation = 'properties.sum',
    getFillColor = [199,233,180],
    getLineColor = [255,255,255],
    pickable=True
)


tooltip = {
    "html": "{properties.navn}<br/>Sum antall: {properties.sum}",
    "style": {
        "backgroundColor": "silver",
        "color": "black",
        "zIndex": "10000",
    },
}

# Render
fig = pydeck.Deck(layers=[layer], 
                          initial_view_state=view_state_norway, 
                          #tooltip=tooltip, 
                          mapbox_key=MAPBOX_TOKEN
                 )
fig.to_html('fig.html')

'/Users/B149899/Projects/deetly/docs/content/pydeck/fig.html'

In [13]:
# Add figure to data package
description = """
## GeoJson layer

```python
layer = pydeck.Layer(
    'GeoJsonLayer',
    json.loads(df_merged.to_json()),
    stroked = False,
    filled = True,
    extruded = True,
    wireframe = True,
    elevationScale = 40,
    getElevation = 'properties.sum',
    getFillColor = [199,233,180],
    getLineColor = [255,255,255],
    pickable=True
)


tooltip = {
    "html": "<b>{properties.navn}</b><br/>Sum antall: {properties.sum}",
    "style": {
        "backgroundColor": "silver",
        "color": "black",
        "zIndex": "10000",
    },
}

# Render
fig = pydeck.Deck(layers=[layer], 
                          initial_view_state=view_state_norway, 
                          tooltip=tooltip, 
                          mapbox_key=MAPBOX_TOKEN)

```

"""

dp.pydeck(fig, 
          "GeoJSON",
          description, 
          showMap=True, 
          #tooltip=json.dumps(tooltip)
         )

In [14]:
tooltip = {
    "html": "<b>{navn}</b><br/>Sum antall: {sum}",
    "style": {
        "backgroundColor": "silver",
        "color": "black",
        "zIndex": "10000",
    },
}

## Scatterplot

In [15]:
source=df_merged[['navn','fylke_navn','sum','coordinates','lat','lng']]
source.head(1)

,navn,fylke_navn,sum,coordinates,lat,lng
0,Oslo,OSLO,38532,"[10.74, 59.98]",59.98,10.74


In [16]:
view_state = pydeck.ViewState(
    longitude = 17,
    latitude= 64,
    zoom= 3,
    minZoom= 3,
    maxZoom= 12,
    pitch= 0,
    bearing= 0)


layer = pydeck.Layer(
    "ScatterplotLayer",
    data=source,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=2,
    radius_min_pixels=1,
    radius_max_pixels=200,
    line_width_min_pixels=1,
    get_position=["lng", "lat"],
    get_radius="sum",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

fig = pydeck.Deck(layers=[layer], tooltip=tooltip, initial_view_state=view_state, mapbox_key=MAPBOX_TOKEN)
fig.to_html('fig.html')

'/Users/B149899/Projects/deetly/docs/content/pydeck/fig.html'

In [17]:
# Add figure to data package
description = """
## Scatterplot layer

```python
view_state = pydeck.ViewState(
    longitude = 17,
    latitude= 64,
    zoom= 3,
    minZoom= 3,
    maxZoom= 12,
    pitch= 0,
    bearing= 0)


layer = pydeck.Layer(
    "ScatterplotLayer",
    data=source,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=2,
    radius_min_pixels=1,
    radius_max_pixels=200,
    line_width_min_pixels=1,
    get_position=["lng", "lat"],
    get_radius="sum",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

fig = pydeck.Deck(layers=[layer], tooltip=tooltip, initial_view_state=view_state, mapbox_key=MAPBOX_TOKEN)
fig.to_html('fig.html')
```

"""

dp.pydeck(fig, "Scatterplot",description, showMap=True, tooltip=json.dumps(tooltip))

## Column layer

In [18]:
layer = pydeck.Layer(
    "ColumnLayer",
    source,
    get_position=["lng", "lat"],
    get_elevation="sum",
    elevation_scale=100,
    radius=5000,
    get_fill_color=[255, 140, 0],
    pickable=True,
    auto_highlight=True,
)

fig_col = pydeck.Deck(
    layer,
    initial_view_state=view_state_norway,
    tooltip=tooltip,
    mapbox_key=MAPBOX_TOKEN
)

fig_col.to_html("fig_col.html")

'/Users/B149899/Projects/deetly/docs/content/pydeck/fig_col.html'

In [19]:
# Add figure to data package
description = """
## Column layer
"""

dp.pydeck(fig_col, "Column",description, showMap=True, tooltip=json.dumps(tooltip))

## GridCell

In [20]:
src = source.copy()

color_lookup = pydeck.data_utils.assign_random_colors(src['fylke_navn'])
src['color'] = src.apply(lambda row: color_lookup.get(row['fylke_navn']), axis=1)

layer = pydeck.Layer(
    "GridCellLayer",
    data=src,
    get_position=["lng", "lat"],
    get_elevation="sum",
    cell_size=10000,
    get_fill_color="color",
    elevation_scale=50,
    pickable=True,
    auto_highlight=True,
)

fig_grid = pydeck.Deck(
    layer,
    initial_view_state=view_state_norway,
    tooltip=tooltip,
    mapbox_key=MAPBOX_TOKEN
)

fig_grid.to_html("fig_grid.html")

'/Users/B149899/Projects/deetly/docs/content/pydeck/fig_grid.html'

In [21]:
# Add figure to data package
description = """
## Gridcell layer

```python
color_lookup = pydeck.data_utils.assign_random_colors(src['fylke_navn'])
src['color'] = src.apply(lambda row: color_lookup.get(row['fylke_navn']), axis=1)

layer = pydeck.Layer(
    "GridCellLayer",
    data=src,
    get_position=["lng", "lat"],
    get_elevation="sum",
    cell_size=10000,
    get_fill_color="color",
    elevation_scale=50,
    pickable=True,
    auto_highlight=True,
)

fig_grid = pydeck.Deck(
    layer,
    initial_view_state=view_state_norway,
    tooltip=tooltip,
    mapbox_key=MAPBOX_TOKEN
)
```
"""

dp.pydeck(fig_grid, "Gridcell",description, showMap=True, tooltip=json.dumps(tooltip))

## Screengrid

In [22]:
view = pydeck.data_utils.compute_view(source[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    'ScreenGridLayer',
    source,
    cell_size_pixels=20,
    color_range=COLOR_RANGE,
    get_position='[lng, lat]',
    pickable=True,
    auto_highlight=True)

fig = pydeck.Deck(layers=[layer], initial_view_state=view,mapbox_key=MAPBOX_TOKEN)
fig.to_html("fig_grid.html")

'/Users/B149899/Projects/deetly/docs/content/pydeck/fig_grid.html'

In [23]:
# Add figure to data package
description = """
## Screen grid

```python
view = pydeck.data_utils.compute_view(source[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    'ScreenGridLayer',
    source,
    cell_size_pixels=20,
    color_range=COLOR_RANGE,
    get_position='[lng, lat]',
    pickable=True,
    auto_highlight=True)

fig = pydeck.Deck(layers=[layer], initial_view_state=view,mapbox_key=MAPBOX_TOKEN)
```

"""

dp.pydeck(fig, "Screen grid",description, showMap=True)

In [24]:
view = pydeck.data_utils.compute_view(src[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    "ColumnLayer",
    data=src,
    get_position=["lng", "lat"],
    get_elevation="sum",
    cell_size=10000,
    radius=5000,
    get_fill_color="color",
    elevation_scale=50,
    pickable=True,
    auto_highlight=True,
)

fig = pydeck.Deck(
    layer,
    initial_view_state=view,
    tooltip=tooltip,
    mapbox_key=MAPBOX_TOKEN
)

fig.to_html("fig.html")

'/Users/B149899/Projects/deetly/docs/content/pydeck/fig.html'

In [25]:
# Add figure to data package
description = """
## Grouped columns layer

```python
view = pydeck.data_utils.compute_view(src[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    "ColumnLayer",
    data=src,
    get_position=["lng", "lat"],
    get_elevation="sum",
    cell_size=10000,
    radius=5000,
    get_fill_color="color",
    elevation_scale=50,
    pickable=True,
    auto_highlight=True,
)

fig = pydeck.Deck(
    layer,
    initial_view_state=view,
    tooltip=tooltip,
    mapbox_key=MAPBOX_TOKEN
)
```

"""

dp.pydeck(fig, "Grouped columns",description, showMap=True, tooltip=json.dumps(tooltip))

## Heatmap

In [26]:
view = pydeck.data_utils.compute_view(source[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    'HeatmapLayer',
    source,
    get_position='[lng, lat]',
    weight="sum",
    pickable=True,
)

fig = pydeck.Deck(layers=[layer], initial_view_state=view,mapbox_key=MAPBOX_TOKEN)
fig.to_html("fig_grid.html")


'/Users/B149899/Projects/deetly/docs/content/pydeck/fig_grid.html'

In [27]:
# Add figure to data package
description = """
## Heatmap layer

```python
view = pydeck.data_utils.compute_view(source[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    'HeatmapLayer',
    source,
    get_position='[lng, lat]',
    weight="sum",
    pickable=True,
)

fig = pydeck.Deck(layers=[layer], initial_view_state=view,mapbox_key=MAPBOX_TOKEN)
```

"""

dp.pydeck(fig, "Heatmap",description, showMap=True)

## Hexagon

In [28]:
coordinates = []
count = 0

for index, row in source.iterrows():
    for index in range(1,row['sum']):
        coordinates.append({'c': row['coordinates']})
        count = count + 1


In [29]:
coordinates[1]

{'c': [10.74, 59.98]}

In [30]:
view = pydeck.data_utils.compute_view(source[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    'HexagonLayer',
    coordinates,
    extruded=True,
    radius=10000,
    elevationScale=1000,
    getPosition="c",
    pickable=True,
)

fig = pydeck.Deck(layers=layer, initial_view_state=view,mapbox_key=MAPBOX_TOKEN)
fig.to_html("fig_grid.html")

'/Users/B149899/Projects/deetly/docs/content/pydeck/fig_grid.html'

In [31]:
# Add figure to data package
description = """
## Hexagon layer

```python
view = pydeck.data_utils.compute_view(source[["lng", "lat"]])
view.pitch = 45
view.bearing = 0

layer = pydeck.Layer(
    'HexagonLayer',
    coordinates,
    extruded=True,
    radius=10000,
    elevationScale=1000,
    getPosition="c",
    pickable=True,
)

fig = pydeck.Deck(layers=layer, initial_view_state=view,mapbox_key=MAPBOX_TOKEN)
```

"""

dp.pydeck(fig, "Hexagon",description, showMap=True)

## Publish

In [32]:
dp.publish()

View: https://public.deetly.com/examples/12e6fabacd0f55e21f231105eecafdca 

Metadata: https://storage.googleapis.com/deetly/examples/12e6fabacd0f55e21f231105eecafdca/datapackage.json 

